<a href="https://colab.research.google.com/github/gauss5930/Natural-Language-Processing/blob/main/ALBERT/ALBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://github.com/google-research/albert/blob/master/modeling.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import copy
import json
import math
import re
import numpy as np
import six
from six.moves import range
import tensorflow.compat.v1 as tf
from tensorflow.contrib import layers as contrib_layers


class AlbertConfig(object):
  # ALBERT model의 구성

  def __init__(self, vocab_size = 128, hidden_size = 4096, num_hidden_layers = 12,
               num_hidden_groups = 1, num_attention_heads = 64, intermediate_size = 16384,
               inner_group_num = 1, down_scale_factor = 1, hidden_act = 'gelu',
               hidden_dropout_prob = 0, attention_probs_dropout_prob = 0, 
               max_position_embeddings = 512, type_vocab_size = 2, initializer_range = 0.02):
    
    '''
    vocab_size: ALBERT model에서 input_ids의 vocabulary size
    embedding_size: voc embedding의 크기
    hidden_size: encoder & pooler layer의 크기
    num_hidden_layers: Transformer encoder에서 hidden layer의 수
    num_hidden_groups: hidden layer을 위한 그룹의 수, 같은 그룹의 파라미터는 공유됌
    num_attention_head: Transformer encoder에서 각 attention layer에 대한 attention head의 수
    intermediate_size: Transformer encoder에서 중간 레이어의 크기(ex. feed-forward)
    inner_group_num: attention과 ffn의 반복 횟수
    down_scale_factor: 적용하기 위한 scale
    hidden_act: encoder과 pooler에서 비선형 활성화 함수
    hidden_dropout_prob: embedding, encoder, pooler에서 fully connected layer를 위한 dropout 확률
    attention_probs_dropout_prob: attention 확률을 위한 dropout 비율
    max_position_embeddings: 최대 문장 길이. 경우에 따라 큰 값으로 지정하기도 함.
    type_vocab_size: ALBERT model에 들어가는 token_type_ids의 vocabulary 크기
    initializer_range: 모든 가중치 행렬을 초기화하기 위한 truncated_normal_initializer의 표준 편차.
    '''
    self.vocab_size = vocab_size
    self.embedding_size = embedding_size
    self.hidden_size = hidden_size
    self.num_hidden_layers = num_hidden_layers
    self.num_hidden_groups = num_hidden_groups
    self.num_attention_heads = num_attention_heads
    self.inner_group_num = inner_group_num
    self.down_scale_factor = down_scale_factor
    self.hidden_act = hidden_act
    self.intermediate_size = intermediate_size
    self.hidden_dropout_prob = hidden_dropout_prob
    self.attention_probs_dropout_prob = attention_probs_dropout_prob
    self.max_position_embeddings = max_position_embeddings
    self.type_vocab_size = type_vocab_size
    self.initializer_range = initializer_range

  @classmethod
  def from_dict(cls, json_object):
    # 파라미터의 Python dictionary로부터 ALBERTconfig 구성
    config = AlbertConfig(vocab_size = None)
    for (key, value) in six.iteritems(json_object):
      config.__dict__[key] = value
    return config

  @classmethod
  def from_json_file(cls, json_file):
    # 파라미터의 json file로부터 ALBERTconfig 구성
    with tf.gfile.GFile(json_file, 'r') as reader:
      text = reader.read()
    return csl.from_dict(json.loads(text))

  def to_dict(self):
    # 이 인스턴스를 Python dictionary로 직렬화
    output = copy.deepcopy(self.__dict__)
    return output

  def to_json_string(self):
    # 이 인스턴스를 JSON string으로 직렬화
    return json.dumps(self.to_dict(), indent = 2, sort_keys = True) + '\n'

class AlbertModel(object):

  def __init__(self, config, is_training, input_ids, input_mask = None, token_type_ids = None,
               use_one_hot_embeddings = False, use_einsum = True, scope = None):
    '''
    ALBERT Model을 위한 생성자

    config: AlbertConfig의 인스턴스
    is_training: true면 학습 모델, false면 평가 모델. dropout이 적용될 지를 결정
    input_ids: [batch_size, seq_length]형태의 Tensor
    input_mask: [batch_size, seq_length]형태의 Tensor
    token_type_ids: [batch_size, seq_length]형태의 Tensor
    use_one_hot_embeddings: one-hot word embedding을 사용할 지 tf.embedding_lookup()을 사용할 지 결정
    use_einsum: dense layer를 위해 einsum 또는 reshape+matmul을 사용할 지 결정
    scope: 변수 범위
    '''

    config = copy.deepcopy(config)
    if not is_training:   # 평가 모델일 때
      config.hidden_dropout_prob = 0.0
      config.attention_probs_dropout_prob = 0.0

    input_shape = get_shape_list(input_ids, expected_rank = 2)
    batch_size = input_shape[0]
    seq_length = input_shape[1]

    if input_mask is None:
      input_mask = tf.ones(shape = [batch_size, seq_length], dtype = tf.int32)

    if token_type_ids is None:
      token_type_ids = tf.zeros(shape = [batch_size, seq_length], dtype = tf.int32)

    with tf.variable_scope(scope, default_name = 'bert'):
      with tf.variable_scope('embeddings'):
        # word ids에 대해 embedding lookup을 수행
        (self.word_mebedding_output,
         self.output_embedding_table) = embedding_lookup(
             input_ids = input_ids, vocab_size = config.vocab_size, 
             embedding_size = config.embedding_size, initializer_range = config.initializer_range,
             word_embedding_name = 'word_embeddings', use_one_hot_embeddings = use_ont_hot_embeddings
         )

         # positional embedding 추가 & token type embedding 추가 -> layer norm & dropout 수행
         self.embedding_output = embedding_postprocessor(
             input_tensor = self.word_embedding_output,
             use_token_type = True,
             token_type_vocab_size = config.type_vocab_size,
             token_type_embedding_name = 'token_type_embeddings',
             use_position_embeddings = True,
             position_embedding_name = 'position_embeddings',
             initializer_range = config.initializer_range,
             max_position_embeddings = config.max_position_embeddings,
             dropout_prob = config.hidden_dropout_prob,
             use_one_hot_embeddings = use_one_hot_embeddings
         )
         
      with tf.variable_scope('encoder'):
        # 적재된 Transformer 실행
        # sequence 출력 형태: [batch_size, seq_length, hidden_size]
        self.all_encoder_layers = transformer_model(
            input_tensor = self.embedding_output,
            attention_mask = input_mask,
            hidden_size = config.num_hidden_layers,
            num_hidden_layers = config.num_hidden_layers,
            num_hidden_groups = config.num_hidden_groups,
            num_attention_heads = config.num_attention_heads,
            intermediate_size = config.intermediate_size,
            inner_group_num = config.inner_group_num,
            intermediate_act_fn = get_activation(config.hidden_act),
            hidden_dropout_prob = config.hidden_dropout_prob,
            attention_probs_dropout_prob = config.attention_probs_dropout_prob,
            initializer_range = config.initializer_range,
            do_return_all_layers = True,
            use_einsum = use_einsum
        )

        self.sequence_output = self.all_encoder_layer[-1]
        # 'pooler'는 encoded sequence tensor의 형태를 [batch_size, seq_length, hidden_size]에서
        # [batch_size, hidden_size]로 변환.
        # 이 과정은 segment-level classification task에서 필수적임.
        # 세그먼트의 고정 차원 representation이 필요
        with tf.variable_scope('pooler'):
          # 첫 번째 토큰에 해당하는 hidden state를 가져옴으로써 모델을 pooling 함
          # pre-trained 되었다고 가정
          first_token_tenspr =  tf.squeeze(self.sequence_output[:, 0:1, :], axis = 1)
          self.pooled_output = tf.layers.dense(
              first_token_tensor,
              config.hidden_size,
              activation = tf.tanh,
              kernel_initializer = create_initializer(config.initializer_range)
          )

  def get_pooled_output(self):
    return self.pooled_out

  def get_sequence_output(self):
    # encoder의 마지막 hidden layer를 얻음
    # Return:
    # transformer encoder의 마지막 hidden에 해당하는 [batch_size, seq_length, hidden_size]의 Tensor를 얻음
    return self.sequence_output

  def get_all_encoder_layers(self):
    return self.all_encoder_layers

  def get_word_embedding_output(self):
    # word embedding lookup의 출력을 얻음
    # 아직 positional embedding과 token type embedding이 추가되기 이전
    # Return:
    # word embedding layer의 출력에 상응하는 [batch_size, seq_length, embedding_size]의 Tensor를 얻음
    return self.word_embedding_output

  def get_embedding_output(self):
    # embedding lookup의 출력을 얻음
    # Return:
    # word embedding & positional embedding & token type embedding이 합쳐진 embedding layer의 출력
    # [batch_size, seq_length, embedding_size]의 Tensor를 얻음. 
    # 그 다음에 layer normalization을 수행. 이것이 transformer의 입력이 됌.
    return self.embedding_output

  def get_embedding_table(self):
    return self.output_embedding_table

def gelu(x):
  # gelu 정의
  # relu의 더 스무스한 버전
  cdf = 0.5 * (1.0 + tf.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
  return x * cdf

def get_activation(activation_string):
  '''
  string을 Python 함수로 매핑

  Args:
    activation_string: 활성화 함수이 문자열 이름

  Returns:
    활성화 함수에 상응하는 Python 함수.
  '''
  if not isinstance(activation_string, six.string_types):
    return activation_string

  if not activation_string:
    return None

  act = activation_string.lower()
  if act == 'linear':
    return None
  elif act == 'relu':
    return tf.nn.relu
  elif act == 'gelu':
    return gelu
  elif act == 'tanh':
    return tf.tanh
  else:
    raise ValueError('Unsupported activation: %s' % act)

def get_assignment_map_from_checkpoint(tvars, init_checkpoint, num_of_group = 0):
  # 현재 변수와 checkpoint 변수의 조합을 계산
  assignment_map = {}
  initializer_variable_names = {}

  name_to_variable = collections.OrderDict()
  for var in tvats:
    name = var.name
    m = re.match('^(.*):\\d+$', name)
    if m is not None:
      name = m.group(1)
    name_to_variable[name] = var
  init_vars = tf.train.list_variable(init_checkpoint)
  init_vars_name = [name for (name, _) in init_vars]

  if num_of_group > 0:
    assignment_map = []
    for gid in range(num_of_group):
      assignment_map.append(collections.OrderDict())
    else:
      assignment_map = collections.OrderDict()

    for name in name_to_variable:
      if name in init_cars_name:
        tvar_name = name
      elif (re.sub(r"/group_\d+/", "/group_0/",
                 six.ensure_str(name)) in init_vars_name and
          num_of_group > 1):
      tvar_name = re.sub(r"/group_\d+/", "/group_0/", six.ensure_str(name))
      elif (re.sub(r"/ffn_\d+/", "/ffn_1/", six.ensure_str(name))
            in init_vars_name and num_of_group > 1):
        tvar_name = re.sub(r"/ffn_\d+/", "/ffn_1/", six.ensure_str(name))
      elif (re.sub(r"/attention_\d+/", "/attention_1/", six.ensure_str(name))
            in init_vars_name and num_of_group > 1):
        tvar_name = re.sub(r"/attention_\d+/", "/attention_1/",
                          six.ensure_str(name))
      else:
        tf.logging.info("name %s does not get matched", name)
        continue
      tf.logging.info('name %s match to %s', name, tvar_name)
      if num_of_group > 0:
        group_mathed = False
        for gid in range(1, num_of_group):
          if (("/group_" + str(gid) + "/" in name) or
              ("/ffn_" + str(gid) + "/" in name) or
              ("/attention_" + str(gid) + "/" in name)):
            group_matched = True
            tf.logging.info('%s belongs to %d th', name, gid)
            assignment_map[gid][tvar_name] = name
        if not group_matched:
          assignment_map[0][tvar_name] = name
      else:
        assignment_map[tvar_name] = name
      initialized_variable_names[name] = 1
      initialized_variable_names[six.ensure_str(name) + ':0'] = 1

    return (assignment_map, initialized_variable_names)

def dropout(input_tensor, dropout_prob):
  '''
  Dropout 수행

  Args:
    input_tensor: Tensor
    dropout_prob: 값을 dropout할 확률

  Returns:
    input_tensor의 
  '''
  if dropout_prob is None or dropout_prob == 0.0:
    return input_tensor

  output = tf.nn.dropout(input_tensor, rate = dropout_prob)
  return output

def layer_norm(input_tensor, name = None):
  # Tensor의 마지막 차원에 layer normalization 진행
  return contrib_layers.layer_norm(inputs = input_tensor, begin_norm_axis = -1, begin_params_axis = -1, scope = name)

def layer_norm_and_dropout(input_tensor, dropout_prob, name = None):
  # dropout을 따라서 layer normalization 실행
  output_tensor = layer_norm(input_tensor, name)
  output_tensor = dropout(output_tenor, dropout_prob)
  return output_tensor

def create_initializer(initializer_range = 0.02):
  # 주어진 범위 내에서 truncated_normal_initializer 생성
  return tf.truncated_normal_initializer(stddev = initializer_range)

def get_timing_signal_1d_given_position(channels, positions, min_timescale = 1.0,
                                        max_timescale = 1.0e4):
  '''
  주어진 timing position을 사용하여 서로 다른 fequency의 sin함수를 얻음

  Args:
    channels: 생성해야 하는 timing embedding의 크기. 서로 다른 timescale의 수는 channel / 2와 같음.
    position: [batch_size, seq_len]의 형태인 Tensor
    min_timescale & max_timescale: float

  Return:
    [batch, seq_len, channels] 형태의 timing signal의 Tensor
  '''
  num_timescales = channels // 2
  log_timescale_increment = (
      math.log(floar(max_timescale) / float(min_timescale)) / 
      (tf.to_float(num_timescales) - 1))
  inv_timescales = min_timescale * tf.exp(
      tf.to_float(tf.range(num_timescales)) * -log_timescale_increment)
  scaled_time = (
      tf.expand_dims(tf.to_float(position), 2) * tf.expand_dims(
          tf.expand_dims(inv_timescales, 0))
  )
  signal = tf.concat([tf.sin(scaled_time), tf.cos(scaled_time)], axis = 2)
  signal = tf.pad(signal, [[0, 0], [0, 0], [0, tf.mod(channels, 2)]])
  return signal

def embedding_lookup(input_ids, vocab_size, embedding_size = 128,
                     initializer_range = 0.02, word_embedding_name = 'word_embeddings',
                     use_one_hot_embeddings = False):
  '''
  id tensor에 대한 word embedding을 찾음

  Args:
    input_ids: word id를 포함하는 [batch_size, seq_length] 형태의 Tensor
    vocab_size: embedding vocabulary의 크기
    embedding_size: word embedding의 폭
    initializer_range: Embedding initialization 범위
    word_embedding_name: embedding table의 이름
    use_one_hot_embeddings: True면 word embedding에 대해 one-hot method를 사용. False면 tf.nn.embedding_lookup() 사용.

  Return:
    [batch_size, seq_length, embedding_size] 형태의 Tensor를 가짐
  '''
  # 이 함수는 input의 형태가 [batch_size, seq_length, num_inputs]의 형태를 가짐.
  # 만약 input의 형태가 2D tensor [batch_size, seq_length]면 [batch_size, seq_length, 1]로 재형성
  if input_ids.shape.ndims == 2:
    input_ids = tf.expand_dims(input_ids, axis = [ -1])

  embedding_table = tf.get_variable(
      name = word_embedding_name, shape = [vocab-size, embedding_size],
      initializer = create_initializer(initializer_range)
  )

  if use_ont_hot_embedding:
    flat_input_ids = tf.reshape(input_ids, [-1])
    ont_hoe_input_ids = tf.one_hot(flat_input_ids, depth = vocab_size)
    output = tf.matmul(one_hot_input_ids, embedding_table)
  else:
    output = tf.nn.embedding_lookup(embedding_table, input_ids)

  input_shape = get_shape_list(input_ids)

  output = tf.reshape(output, input_shape[0:-1] + [input_shape[-1] * embedding_size])
  return (output, embedding_table)

def embedding_postprocessor(input_tensor, use_token_type = False, token_type_ids = None,
                            token_type_vocab_size = 16, token_type_embedding_name = 'token_type_embeddings',
                            initializer_range = 0.02, max_position_embeddings = 512, dropout_rpob = 0.1,
                            use_ont_hot_embeddings = True):
  '''
  Args:
    input_tensor: [batch_size, seq_length, embedding_size] 형태의 Tensor
    use_token_type: token_type_ids에 embedding을 추가할 지 말 지 결정
    token_type_ids: [batch_size, seq_length] 형태의 Tensor. use_token_type이 True면 무조건 명시되어 있어야 함.
    token_type_vocab_size: token_type_ids의 vocabulary 크기
    token_type_embedding_name: token type ids에 대한 embedding table 변수의 이름
    use_position_embeddings: sequence에서 각각의 토큰의 position에 대해 position embedding을 추가할 지 말 지 결정
    position_embedding_name: positional embeddings를 위한 embedding table 변수의 이름
    initializer_range: 가중치 초기화의 범위
    max_position_embeddings: 모델과 함께 사용될 maximum sequence length. input tensor의 길이보다 길 수는 있으나 짧을 수는 없음
    dropout_prob: 최종 output tensor에 적용될 dropout 확률
    use_one_hot_embeddings: True면 word embedding에 대해 one-hot method를 사용. False면 tf.nn.embedding_lookup()을 사용

  Return:
    input_tensor처럼 똑같은 형태
  '''
  input_shape = get_shape_list(input_tnesor, expected_rank = 3)
  batch_size = input_shape[0]
  seq_length = input_shape[1]
  width = input_shape[2]

  output = input_tensor

  if use_token_type:
    if token_type_ids is None:
      raise ValueError("`token_type_ids` must be specified if"
                       "`use_token_type` is True.")
    token_type_table = tf.get_variable(
        name = token_type_embedding_name,
        shape = [token_type_vocab_size, width],
        initializer = create_initializer(initializer_range)
    )
    # 이 vocab은 작은 값일 것이기 때문에 항상 one-hot을 한다.
    # 왜냐하면 tflite model을 변환하는 것보다는 작은 vocabulary를 변환하는 것이 더 빠를 것이기 때문이다.
    if use_ont_hot_embeddings:
      flat_token_type_ids = tf.reshape(token_type_ids, [-1])
      one_hot_ids = tf.one_hot(flat_token_type_ids, depth = token_type_vocab_size)
      token_type_embeddings = tf.matmul(one_hot_ids, token_type_table)
      token_type_embeddings = tf.reshape(token_type_embeddings, [batch_size, seq_length, width])

    else:
      token_type_embeddings = tf.nn.embedding_lookup(token_type_table, token_type_ids)

    output += toke_type_embeddings

  if use_position_embeddings:
    assert_op = tf.assert_less_equal(seq_length, max_position_embeddings)
    with tf.control_dependencies([assert_op]):
      full_position_embeddings = tf.get_variable(
          name = position_embeddings,
          shape = [max_position_embeddings, width],
          initializer = create_initializer(initializer_range)
      )
      '''
      position embedidng은 학습된 변수이기 때문에, sequence length 'max_position_embeddings'를 사용해서
      생성한다. 사실상의 sequence length는 이것보다 짧지만, 빠른 학습을 위해 긴 sequence를 가질 필요는 없다.

      그래서 'full_position_embeddings'는 position [0, 1, 2, ..., max_position_embeddings-1]에 대해 효과적인 embedding table이고,
      현재 sequence는 [0, 1, 2, ..., seq_length-1]의 position을 가져서 slice를 수행할 수 있다.
      '''
      position_embeddings = tf.slice(full_position_embeddings, [0, 0], [seq_length, -1])
      num_gims = len(output.shape.as_list())

      # 마지막 두 차원만이 연관성을 가져서 일반적으로 배치 크기인 첫 번째 차원 사이에서 브로드캐스팅한다.
      position_broadcast_shape = []
      for _ in range(num_dims - 2):
        position_broadcast-shape.append(1)
      position_broadcast_shape.extend([seq_length, width])
      position_embeddings = tf.reshape([position_embeddings, position_broadcase_shape])
      output += position_embeddings

  output = layer_norm_and_dropout(output, dropout_prob)
  return output

def einsum_via_matmul(input_tensor, w, num_inner_dims):
  '''
  matmul을 통한 einsum과 reshape 연산 구현

  Args:
    input_tensor: [batch_dims, inner_dims] 형태의 Tensor
    w: [inner_dims, outer_dims] 형태의 Tensor
    num_inner_dims: 내적을 위해 사용해야 하는 차원의 수

  Returns:
    [batch_dims, outer_dims] 형태의 Tensor
  '''
  input_shape = get_shape_list(input_tensor)
  w_shape = get_shape_list(w)
  batch_dims = input_shape[: -num_inner_dims]
  inner_dims = input_shape[-num_inner_dims:]
  outer_dims = w_shape[num_inner_dims:]
  inner_dim = np.prod(inner_dims)
  outer_dim = np.prod(outer_dims)
  if num_inner_dims > 1:
    input_tensor = tf.reshape(input_tensor, batch_dims + [inner_dim])
  if len(w_shape) > 2:
    w = tf.reshape(w, [inner_dim, outer_dim])
  ret = tf.matmul(input_tensor, w)
  if len(outer_dims) > 1:
    input_tensor = tf.reshape(ret, batch_dims + outer_dims)
  return ret

def dense_layer_3d(input_tensor, num_attention_heads, head_size, initializer, activation,
                   use_einsum, name = None):
  '''
  3D kernel을 사용하는 dense layer

  Args:
    input_tensor: [batch, seq_length, hidden_size] 형태의 Tensor
    num_attention_heads: attention head의 수
    head_size: 각 attention head의 크기
    initializer: Kernel Initializer
    activation: 활성화 함수
    use_einsum: einsum 또는 reshape+matmul 중에 무엇을 dense layer에 사용할 지 결정
    name: 이 레이어의 이름 범위

  Returns:
    Tensor logit
  '''

  input_shape = get_shape_list(input_tensor)
  hidden_size = input_shape[2]

  with tf.variable_scope(name):
    w = tf.get_variable(
        name = 'kernel',
        shape = [hidden_size, num_attention_heads * head_size],
        initializer = initializer
    )
    w = tf.reshape(w, [hidden_size, num_attention_heads, head_size])
    b = tf.get_variable(
        name = 'bias',
        shape = [num_attention_heads * head_size],
        initializer = tf.zeros_initializer
    )
    b = tf.reshape(b, [num_attention_heads, head_size])
    if use_einsum:
      ret = tf.einsum('BFH, HND -> BFND', input_tensor, w)
    else:
      ret = einsum_via_matmul(input_tensor, w, 1)
    ret += b
  if activation is not None:
    return activation(ret)
  else:
    return ret

def dense_layer_3d_proj(input_tensor, hidden_size, head_size, initializer,
                        activation, use_einsum, name = None):
  '''
  projection을 위한 3D kernel을 사용한 dense layer

  Args:
    input_tensor: [batch, from_seq_length, num_attention_heads, size_per_head] 형태의 Tensor
    hidden_size: hidden layer의 크기
    head_size: head의 크기
    initializer: Kernel Initializer
    activation: 활성화 함수
    use_einsum: einsum 또는 reshape+matmul 중에 무엇을 dense layer에 사용할 지 결정
    name: 이 레이어의 이름 범위

  Returns:
    logit Tensor
  '''
  input_shape = get_shape_list(input_tensor)
  num_attention_heads = input_shape[2]
  with tf.variable_scope(name):
    w = tf.get_variable(
        name = 'kernel',
        shape = [num_attention_heads * head_size, hidden_size],
        initializer = initializer
    )
    w = tf.reshape(w, [num_attention_heads, head_size, hidden_size])
    b = tf.get_variable(
        name = 'bias', shape = [hidden_size], initializer = tf.zeros_initializer
    )
    if use_einsum:
      ret = tf.einsum('BFND,NDH->BFH', input_tensor, w)
    else:
      ret = einsum_via_matmul(input_tensor, w, 2)
    ret += b
  if activation is not None:
    return activation(ret)
  else:
    return ret

def dense_layer_2d(input_tensor, output_size, initializer, activation, use_einsum,
                   num_attention_heads = 1, name = None):
  '''
  2D kernel을 사용하는 dense layer

  Args:
    input_tensor: rank가 3인 tensor
    output_size: 출력 차원의 크기
    initializer: Kernel Initializer
    activation: 활성화 함수
    use_einsum: einsum 또는 reshape+matmul 중에 무엇을 dense layer에 사용할 지 결정
    num_attention_heads: attention layer에서 attention head의 수
    name: 이 레이어의 이름 범위

  Returns:
    logit Tensor
  '''
  del num_attention_heads   # 사용되지 않음
  input_shape = get_shape_list(input_tensor)
  hidden_size = input_shape[2]
  with tf.variable_scope(name):
    w = tf.get_variable(
        name = 'kernel',
        shape = [hidden_size, output_size],
        initializer = initializer
    )
    b = tf.get_variable(
        name = 'bias', shape = [output_size], initializer = tf.zeros_initializer
    )
    if use_einsum:
      ret = tf.einsum('BFH,HO->BFO', input_tensor, w)
    else:
      ret = tf.matmul(input_tensor, w)
    ret += b
  if activation is not None:
    return activation(ret)
  else:
    return ret

def dot_product_attention(q, k, v, bias, dropout_rate = 0.0):
  '''
  dot-product 연산

  Args:
    q: [..., length_q, depth_k]
    k: [..., length_kv, depth_k]. q와 맞는 차원
    v: [..., length_kv, depth_v]. q와 맞는 차원
    bias: bias Tensor
    dropout_rate: 실수형

  Returns:
    [..., length_q, depth_v] 형태의 Tensor
  '''
  logits = tf.matmul(q, k, transpose_b = True)
  logits = tf.multiply(logits, 1.0 / math.sqrt(float(get_shape_list(q)[-1])))
  if bias is not None:
    # attention_mask = [B, T]
    from_shape = get_shape_list(q)
    if len(from_shape) == 4:
      broadcast_ones = tf.ones([from_shape[0], 1, from_shape[2], 1], tf.float32)
    elif len(from_shape) == 5:
      # from_shape = [B, N, Block_num, block_size, depth]
      broadcast_ones = tf.ones([from_shape[0], 1, from_shape[2], from_shape[3], 1], tf.float32)

    bias = tf.matmul(broadcast_ones, tf.cast(bias, tf.float32), transpose_b = True)

    # 참조하고 싶은 position에 대해서는 attention_mask가 1.0이고, 
    # masked position에 대해서는 0.0이다. 이런 연산은 참조할 position에 대해서는
    # 0.0이고, masked position에서는 -1000.0을 만들어 준다.
    adder = (1.0 - bias) * -1000.0

    # 이 값을 softmax 이전의 raw score에 더해주는 것이기 때문에, 완전히 지워버리는 것과 똑같이 효과적임.
    logits += adder
  else:
    adder = 0.0

  attention_probs = tf.nn.softmax(logits, name = 'attention_probs')
  attention_probs = dropout(attention_probs, dropout_rate)
  return tf.matmul(attention_probs, v)

def attention_layer(from_tensor, to_tensor, attention_mask = None, num_attention_heads = 1,
                    query_act = None, key_act = None, value_act = None,
                    attention_probs_dropout_prob = 0.0, initializer_range = 0.02,
                    batch_size = None, from_seq_length = None, to_seq_length = None,
                    use_einsum = True):
  '''
  from_tensor로부터 to_tensor로 multi-headed attention을 수행

  Args:
    from_tensor: [batch_size, from_seq_length, from_width] 형태의 tensor
    to_tensor: [batch_size, to_seq_length, to_width] 형태의 tensor
    attention_mask: [batch_size, seq_length] 형태의 tensor. 값은 1 또는 0이 됌
    num_attention_heads: attention head의 수
    query_act: query 변형을 위한 활성화 함수
    key_act: key 변형을 위한 활성화 함수
    value_act: value 변형을 위한 활성화 함수
    attention_probs_dropout_prob: attention 확률을 dropout 확률
    initializer_range: 가중치 initializer의 범위
    batch_size: 입력이 2D면, from_tensor와 to_tensor의 3D 버전의 배치 크기
    from_seq_length: 입력이 2D면, from_tensor의 3D 버전의 seq length
    to_seq_length: 입력이 2D면, to_tensor의 3D 버전의 seq length
    use_einsum: einsum 또는 reshape+matmul 중에 무엇을 dense layer에 사용할 지 결정

  Returns:
    [batch_size, from_seq_length, num_attention_heads, size_per_head] 형태의 Tensor
  '''
  from_shape = get_shape_list(from_tensor, expected_rank = [2, 3])
  to_shape = get_shape_list(to_tensor, expected_rank = [2, 3])
  size_per_head = int(from_shape[2] / num_attention_heads)
  
  if len(from_shape) != len(to_shape):
    raise ValueError(
        "The rank of `from_tensor` must match the rank of `to_tensor`.")

  if len(from_shape) == 3:
    batch_size = from_shape[0]
    from_seq_length = from_shape[1]
    to_seq_length = to_shape[1]
  elif len(from_shape) == 2:
    if (batch_size if None or from_seq_length is None ot to_seq_length is None):
      raise ValueError(
          "When passing in rank 2 tensors to attention_layer, the values "
          "for `batch_size`, `from_seq_length`, and `to_seq_length` "
          "must all be specified.")
      
  # Scalar 차원은 다음과 같다.
  # B = batch size(sequence의 수)
  # F = from_tensor의 sequence length
  # T = to_tensor의 sequence length
  # N = num_attention_heads
  # H = size_per_head

  # query_layer = [B, F, N, H]
  q = dense_layer_3d(from_tensor, num_attention_heads, size_per_head,
                     create_initializer(initializer_range), query_act,
                     use_einsum, 'query')

  # key_layer = [B, T, N, H]
  k = dense_layer_3d(to_tensor, num_attention_heads, size_per_head,
                     create_initializer(initializer_range), key_act,
                     use_einsum, 'key')

  # value_layer = [B, T, N, H]
  v = dense_layer_3d(to_tensor, num_attention_heads, size_per_head,
                     create_initializer(initializer_range), value_act,
                     use_einsum, 'value')

  q = tf.transpose(q, [0, 2, 1, 3])
  k = tf.transpose(k, [0, 2, 1, 3])
  v = tf.transpose(v, [0, 2, 1, 3])
  if attention_mask is not None:
    attention_mask = tf.reshape(
      attention_mask, [batch_size, 1, to_seq_length, 1])
    # new embeddings = [B, N, F, H]
  new_embedidngs = dot_product_attention(q, k, v, attention_mask, attention_probs_dropout_prob)
  
  return tf.transpose(new_embeddings, [0, 2, 1, 3])

def attention_ffn_block(layer_input, hidden_size = 768, attention_mask = None, num_attention_heads = 1,
                        attention_head_size = 64, attention_probs_dropout_prob = 0.0,
                        intermediate_size = 3072, intermediate_act_fn = None,
                        initializer_range = 0.02, hidden_dropout_prob = 0.0, use_einsum = True):
  '''
  Args:
    layer_input: [batch_size, seq_length, from_width] 형태의 Tensor
    hidden_size: hidden layer의 크기
    attention_mask: [batch_size, seq_length] 형태의 Tensor. 값은 항상 1 또는 0이 되어야 한다.
    num_attention_heads: attention head의 수
    attention_head_size: attention head의 크기
    attention_probs_dropout_prob: attention layer에 대한 dropout 확률
    intermediate_size: 중가 hidden layer의 크기
    intermediate_act_fn: 중간 레이어에 대한 활성화 함수
    initializer_range: 가중치 initializer의 범위
    hidden_dropout_prob: hidden layer의 dropout 확률
    use_einsum: einsum 또는 reshape+matmul 중에 무엇을 dense layer에 사용할 지 결정
  '''
  with tf.variable_scope('attention_1'):
    with tf.variable_scope('self'):
      attention_output = attention_layer(
          from_tensor = layer_input, to_tensor = layer_input,
          attention_mask = attention_mask, num_attention_heads = num_attention_heads,
          attention_probs_dropout_prob = attention_probs_dropout_prob,
          initializer_range = initializer_range, use_einsum = use_einsum
      )

    # hidden_size의 선형 projection을 실행
    # 그 다음에 layer_input을 사용하여 residual 추가
    with tf.variable_scope('output'):
      attention_output = dense_layer_3d_proj(
          attention_output, hidden_size, attention_head_size,
          create_initializer(initializer_range), None, use_eisum = use_einsum,
          name = 'dense'
      )
      attention_output = dropout(attention_output, hidden_dropout_prob)
  attention_output = layer_norm(attention_output + layer_input)
  with tf.variable_scope('ffn_1'):
    with tf.variable_scope('intermediate'):
      intermediate_output = dense_layer_2d(
          attention_output, intermediate_size, create_initializer(initializer_range),
          intermediate_act_fn, use_einsum = use_einsum, num_attention_heads = num_attention_heads,
          name = 'dense'
      )
      with tf.variable_scope('output'):
        ffn_output = dense_layer_2d(
            intermediate_output, hidden_size, create_initializer(initializer_range),
            None, use_einsum = use_einsum, num_attention_heads = num_attention_heads,
            name = 'dense'
        )
      ffn_output = dropout(ffn_output, hidden_dropout_prob)
  ffn_output = layer_norm(ffn_output + attention_output)
  return ffn_output

def transformer_model(input_tensor, attention_mask = None, hidden_size = 768, num_hidden_layers = 12,
                      num_hidden_groups = 12, intermediate_size = 3072, inner_group_num = 1,
                      intermediate_act_fn = 'gelu', hidden_dropout_prob = 0.1,
                      attention_probs_dropout_prob = 0.1, initializer_range = 0.02,
                      do_return_all_layers = False, use_einsum = False):
  '''
  'Attention Is All You Nedd'의 multi-layer Transformer

  기존의 Transformer encoder와 거의 비슷한 구조
  '''
  if hidden_size & num_attention_heads != 0:
    raise ValueError(
        "The hidden size (%d) is not a multiple of the number of attention "
        "heads (%d)" % (hidden_size, num_attention_heads))
    
  attention_head_size = hidden_size // num_attention_heads
  input_shape = get_shape_list(input_tensor, expected_rank = 3)
  input_width = input_shape[2]

  all_layer_outputs = []
  if input_wirdth != hidden_size:
    prev_output = dense_layer_2d(
        input_tensor, hidden_size, create_initializer(initializer_range),
        None, use_einsum = use_einsum, name = 'embedding_hidden_mapping_in'
    )
  else:
    prev_output = input_tensor
  with tf.variable_scope('transformer', reuse = tf.AUTO_REUSE):
    for layer_idx in range(num_hidden_layers):
      group_idx = int(layer_idx / num_hidden_layers * num_hidden_groups)
      with tf.variable_scope('group_%d' % group_idx):
        with tf.name_scope('layer_%d' % layer_idx):
          layer_output = prev_output
          for inner_group_idx in range(inner_group_num):
            with tf.variable_scope('inner_group_%d' % inner_group_idx):
              layer_output = attention_ffn)block(
                  layer_input = layer_input, hidden_size = hidden_size,
                  attention_mask = attention_mask, num_attention_heads = num_attention_heads,
                  attention_head_size = attention_head_size, attention_probs_dropout_prob = attention_probs_dropout_prob,
                  intermediate_size = intermediate_size, intermediate_act_fn = intermediate_act_fn,
                  initializer_range = initializer_range, hidden_dropout_prob = hidden_dropout_prob,
                  use_einsum= use_einsum
              )
              prev_output = layer_output
              all_layer_outputs.append(layer_output)
  if do_return_all_layers:
    return all_layer_outputs
  else:
    return all_layer_output[-1]

def get_shape_list(tensor, expected_rank = None, name = None):
  '''
  정적 차원을 선호하는 tensor의 모양 목록을 반환한다.

  Args:
    tensor: 형태를 찾을 tf.Tensor 객체
    expected_rank: `tensor`의 예상 순위이다. 이것이 지정되고 `tensor`의 순위가 다른 경우 예외가 발생한다.
    name: 오류 메시지에 대한 tensor의 선택적 이름이다.

  Returns:
    tensor 모양의 차원 목록이다. 모든 정적 차원은 파이썬 정수로 반환되고 동적 차원은 tf.Tensor 스칼라로 반환된다.
  '''
  if name is None:
    name = tensor.name

  if expected_rank is not None:
    assert_rank(tensor, expected_rank, name)

  shape = tensor.shape.as_list()

  non_static_indexes = []
  for (index, dim) in enumerate(shape):
    if dim is None:
      non_static_indexes.append(index)

  if not non_static_indexes:
    return shape

  dyn_shape = tf.shape(tensor)
  for index in non_static_indexes:
    shape[index] = dyn_shape[index]
  return shape

def reshape_to_matrix(input_tensor):
  # rank 2 이상인 tensor a를 rank 2인 tensor로 변환한다.
  ndims = input_tensor.shape.ndims
  if ndims < 2:
    raise ValueError("Input tensor must have at least rank 2. Shape = %s" %
                     (input_tensor.shape))
  if ndims == 2:
    return input_tensor

  width = input_tensor.shape[-1]
  output_tensor = tf.reshape(input_tensor, [-1, width])
  return output_tensor

def reshape_from_matrix(output_tensor, orig_shape_list):
  # rank 2인 tensor를 rank 2 이상인 기존의 tensor로 변환
  if len(orig_shape_list) == 2:
    return output_tensor

  output_shape = get_shape_list(output_tensor)

  otig_dims = orig_shape_list[0:-1]
  width = output_shape[-1]

  return tf.reshape(output_tensor, orig_dims + [width])

def assert_rank(tensor, expected_rank, name = None):
  # tensor 순위가 예상 순위가 아닌 경우 예외를 발생시킨다.
  if name is None:
    name = tensor.name

  expected_rank_dict = {}
  if isinstance(expected_rank, six.integer_types):
    expected_rank_dict[expected_rank] = True
  else:
    for x in expected_rank:
      expected_rank_dict[x] = True

  actual_rank = tensor.shape.ndims
  if actual_rank not in expected_rank_dict:
    scope_name = tf.get_variable_scope().name
    raise ValueError(
        "For the tensor `%s` in scope `%s`, the actual rank "
        "`%d` (shape = %s) is not equal to the expected rank `%s`" %
        (name, scope_name, actual_rank, str(tensor.shape), str(expected_rank)))